## Progressive Growing of GANs

By now, you probably have realized how difficult it can be to train GANs. They are fairly unstable, especially when trying to generate high-dimensional samples, such as high-resolution images! 

However, researchers never lack ideas to improve them and this 2017 paper made trainings of GANs more stable: *Progressive growing of GANs for improved quality, stability and variation*.

The main idea behind this paper is the following: since training GANs on smaller images is easier, we can progressively grow the network and the generated images' dimensions to make training easier for the network. It is illustrated by the figure below:

<img src='assets/progan2.png' width=70% />


### Layer fading 

Each level, or depth, is training for a certain number of epochs (e.g., 10 epochs). Then a new layer is added in the discriminator and the generator and we start training with these additional layers. However, when a new layer is added, it is faded in smoothly, as described by the following figure:

<img src='assets/layer_fading2.png' width=70% />

The `toRGB` and `fromRGB` layers are the layers projecting the feature vector to the RGB space (HxWx3) and the layer doing the opposite, respectively. 

Let's look at the example:
* **(a)** The network is currrently training at 16x16 resolution, meaning that the generated images are 16x16x3
* **(b)** We are adding two new layers to train at 32x32 resolution. However, we are fading in the new layers by doing the following:
    * For the generator, we take the output of the 16x16 layer and use nearest neighbor image resize to double its resolution to 32x32. The same output will also be fed to the 32x32 layer. Then we calculate the output of the network by doing a weighted sum of $(1- \alpha)$ the upsampled 16x16 image and $\alpha$ the 32x32 layer output. 
    * For the discriminator, we do something similar but to reduce the resolution, we use an average pooling layer
    * The network trains for N epochs at each resolution. During the first $N/2$ epochs, we start with $/alpha = 0$ and increase alpha linearly to $/alpha = 1$. Then we train for the remaining $N/2$ epochs with $/alpha = 1$.
* **(c)** The network is now training at 32x32 resolution

#### Exercise

In this exercise, you will implement the Generator of the ProGan model. To make your life easier, I already implemented two torch modules: `GeneratorFirstBlock` and `GeneratorBlock`. 
* The `GeneratorFirstBlock` module takes the the latent vector as input and outputs a multi-dimensional feature map
* The `GeneratorBlock` module corresponds to each layer added when increasing the resolution

**Note:** In the paper, the authors are using a new type of normalization, called PixelNormalization. I encourage you to read the paper but for the sake of simplicity, I did not add any normalization here. 

In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class GeneratorFirstBlock(nn.Module):
    """
    This block follows the ProGan paper implementation.
    Takes the latent vector and creates feature maps.
    """
    def __init__(self, latent_dim: int):
        super(GeneratorFirstBlock, self).__init__()
        # initial block 
        self.conv0 = nn.ConvTranspose2d(latent_dim, 512, kernel_size=4)
        self.conv1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x: torch.Tensor):
        # x is a (batch_size, latent_dim) latent vector, we need to turn it into a feature map
        x = torch.unsqueeze(torch.unsqueeze(x, -1), -1)
        x = self.conv0(x)
        x = self.activation(x)
        
        x = self.conv1(x)
        x = self.activation(x)
        return x

In [ ]:
class GeneratorBlock(nn.Module):
    """
    This block follows the ProGan paper implementation.
    """
    def __init__(self, in_channels: int, out_channels: int):
        super(GeneratorBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x: torch.Tensor):
        x = interpolate(x, scale_factor=2)
        x = self.conv1(x)
        x = self.activation(x)

        x = self.conv2(x)
        x = self.activation(x)
        return x

Using the above two blocks, you can implement the Generator module. The end resolution that we want to reach is 512x512 and we will start at a 4x4 resolution. 


#### init
The `__init__` method should contain enough blocks to work at full resolution. We are only instantiating the generator once! So you will need to:
* Create one GeneratorFirstBlock module
* Create enough GeneratorBlocks modules such that the final resolution is 512x512
* Create one `toRGB` layer per resolution. 

The number of filters in each layer is controlled by the `num_filters` function below.


#### forward

The forward method does the following:
* Takes the latent vector, the current resolution and `alpha` as input 
* Runs the latent vector through the different blocks and performs `alpha` fading


In the original paper, the number of filters of convolution layers increases with depth. The `num_filters` function below will help you progammatically increase the number of filters based on the stage (or depth) of the generator. A depth of 1 correspond to 4x4 resolution, a depth of 2 to an 8x8 resolution etc. 

* you can the torch `interpolate` function to double the resolution of an image
* you can use the `np.log2` function to map the resolution of the input image to a "depth" (or stage) level. For example, `np.log2(512) = 9` and `np.log2(4)` = 2.
* when training at 4x4 resolution, you should not perform $\alpha-$fading.
</details>


In [ ]:
import tests

from torch.nn.functional import interpolate

In [ ]:
def num_filters(stage: int, 
                fmap_base: int = 8192,
                fmap_decay: float = 1.0,
                fmap_max: int = 512): 
    """
    A small helper function to computer the number of filters for conv layers based on the stage/depth,
    stage = log2(resolution)
    From the original repo https://github.com/tkarras/progressive_growing_of_gans/blob/master/networks.py#L252
    """
    return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)

In [ ]:
class Generator(nn.Module):
    """
    Generator: takes a latent vector as input and output an image.
    args:
    - max_resolution: max image resolution
    - latent_dim: dimension of the input latent vector
    """
    def __init__(self, max_resolution: int, latent_dim: int):
        super(Generator, self).__init__()
        # following the original implementation
        resolution_log2 = int(np.log2(max_resolution))
        ####
        # IMPLEMENT HERE
        ####

    def forward(self, x: torch.Tensor, current_res: int, alpha: float = 1.0):
        """
        args:
        - x: latent vector (N, latent_dim)
        - current_res: current resolution (eg, 4, 8, 64, 512 etc)
        - alpha: fading coefficient
        """
        resolution_log2 = int(np.log2(current_res))
        ####
        # IMPLEMENT HERE
        
        ####
        return images_out

In [ ]:
generator = Generator(max_resolution=512, latent_dim=128)

In [ ]:
tests.check_progan_generator(generator)